In [3]:
import cv2
import os
import numpy as np
import os.path as osp
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.nn import Module
import torch.nn.functional as F
from torch.nn.modules.padding import ReplicationPad2d
import glob
# from tensorboardX import SummaryWriter
from torch.utils.tensorboard import SummaryWriter
import re
import mylib.polSAR_utils as psr
from mylib import types
import torchvision
import yaml
from mylib import types
from typing import Optional
import random
from torch import Tensor
from random import shuffle

In [19]:
a = [1,2]
shuffle(a)
a

[1, 2]

In [40]:
a = list(range(10))
# a.pop([1, 3, 5])
a

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [33]:
random.sample(range(10), 5)

[9, 2, 0, 5, 6]

In [23]:
path = r'data/GF3/data'
# path = r'/data/csl/PolSAR_building_det/data/GF3'
all_files = []
for root, dirs, files in os.walk(path):
    if root.endswith('Hoekman'):
        for dir in dirs:
            all_files.append(osp.join(root, dir))


In [28]:
all_files

624/Hoekman/19',
 'data/GF3/data/E132_N34_日本安芸/升轨/1/20170624/Hoekman/100',
 'data/GF3/data/E132_N34_日本安芸/升轨/1/20170624/Hoekman/112',
 'data/GF3/data/E132_N34_日本安芸/升轨/1/20170624/Hoekman/29',
 'data/GF3/data/E132_N34_日本安芸/升轨/1/20170624/Hoekman/12',
 'data/GF3/data/E132_N34_日本安芸/升轨/1/20170624/Hoekman/72',
 'data/GF3/data/E132_N34_日本安芸/升轨/1/20170624/Hoekman/45',
 'data/GF3/data/E132_N34_日本安芸/升轨/1/20170624/Hoekman/95',
 'data/GF3/data/E132_N34_日本安芸/升轨/1/20170624/Hoekman/3',
 'data/GF3/data/E132_N34_日本安芸/升轨/1/20170624/Hoekman/97',
 'data/GF3/data/E132_N34_日本安芸/升轨/1/20170624/Hoekman/13',
 'data/GF3/data/E132_N34_日本安芸/升轨/1/20170624/Hoekman/40',
 'data/GF3/data/E132_N34_日本安芸/升轨/1/20170624/Hoekman/83',
 'data/GF3/data/E132_N34_日本安芸/升轨/1/20170624/Hoekman/88',
 'data/GF3/data/E132_N34_日本安芸/升轨/1/20170624/Hoekman/101',
 'data/GF3/data/E132_N34_日本安芸/升轨/1/20170624/Hoekman/32',
 'data/GF3/data/E132_N34_日本安芸/升轨/1/20170624/Hoekman/132',
 'data/GF3/data/E132_N34_日本安芸/升轨/1/20170624/Hoekman/69',
 'data/GF3/

In [27]:
len(all_files)*0.01


42.61

In [27]:
c = a()
c._a__b
setattr(c, '_a__b', 5)
c._a__b

5

In [3]:
a = torch.tensor([1+2j, 3+4j]).view(1,1,1,1,2)
a, a.abs()

(tensor([[[[[1.+2.j, 3.+4.j]]]]]), tensor([[[[[2.2361, 5.0000]]]]]))

In [13]:
np.angle(np.array([100,1, 1+1j, 10000j, -1, -1-1j, -1j, 3-3j]).reshape(1,1,1,1,1,2,-1))

array([[[[[[[ 0.        ,  0.        ,  0.78539816,  1.57079633],
            [ 3.14159265, -2.35619449, -1.57079633, -0.78539816]]]]]]])

In [18]:

class ComplexUpsample(Module):
    def __init__(self, size = None, scale_factor = None,
                 mode: str = 'nearest', align_corners: Optional[bool] = None) -> None:
        super().__init__()
        self.up = nn.Upsample(size=size, scale_factor=scale_factor, mode=mode, align_corners=align_corners)
        
    def forward(self, input: Tensor) -> Tensor:
        if 'complex' not in str(input.dtype):
            raise ValueError('not support non-complex data')
        real = self.up(input.real)
        imag = self.up(input.imag)
        return real + 1j*imag

In [20]:
a = torch.tensor([[1,2],[3,4]])+1j*torch.tensor([[8,6],[7,5]]).view(1,1,2,2)
b = ComplexUpsample(scale_factor=2, mode='bilinear')
a,b(a)

(tensor([[[[1.+8.j, 2.+6.j],
           [3.+7.j, 4.+5.j]]]]),
 tensor([[[[1.0000+8.0000j, 1.2500+7.5000j, 1.7500+6.5000j, 2.0000+6.0000j],
           [1.5000+7.7500j, 1.7500+7.2500j, 2.2500+6.2500j, 2.5000+5.7500j],
           [2.5000+7.2500j, 2.7500+6.7500j, 3.2500+5.7500j, 3.5000+5.2500j],
           [3.0000+7.0000j, 3.2500+6.5000j, 3.7500+5.5000j, 4.0000+5.0000j]]]]))

In [16]:
str(a.dtype)

'torch.complex64'